In [2]:
from google.colab import files
import io

uploaded = files.upload()

Saving twitter_training.csv to twitter_training.csv
Saving twitter_validation.csv to twitter_validation.csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import torch
from transformers import AutoTokenizer
import torch.nn as nn

# Twitter sentiment analysis


random/chance = 25% accuracy
- sentiment - multiclass:
    - positive = 3
    - neutral = 2
    - irrelevant = 1
    - negative = 0

### Project 1: simple sentiment analysis - logistic regression with tokenization
predicting sentiment based on tweet

Data structure:
- tokenise tweets
    

### Project 2: simple sentiment analysis - logistic regression with tokenization and word embedding

### Project 3: simple sentiment analysis - logistic regression with tokenization, word embedding and scaled features

### Project 4: XGBOOST

### Project 5: SVM

  
### Project : aspect-based sentiment analysis
predicting sentiment based on entity and tweets


# LOAD DATA

In [4]:
df = pd.read_csv(io.BytesIO(uploaded['twitter_training.csv']))
df.head()

,id,entity,sentiment,tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2402,Borderlands,Positive,So I spent a few hours making something for fu...
2,2403,Borderlands,Neutral,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM..."
3,2404,Borderlands,Positive,that was the first borderlands session in a lo...
4,2405,Borderlands,Negative,the biggest dissappoinment in my life came out...


In [5]:
df_test = pd.read_csv(io.BytesIO(uploaded['twitter_validation.csv']))
df_test.head()

,id,entity,sentiment,tweet
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [6]:
df_join = pd.concat([df,df_test], ignore_index=True).reset_index(drop=True)
df_join

,id,entity,sentiment,tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2402,Borderlands,Positive,So I spent a few hours making something for fu...
2,2403,Borderlands,Neutral,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM..."
3,2404,Borderlands,Positive,that was the first borderlands session in a lo...
4,2405,Borderlands,Negative,the biggest dissappoinment in my life came out...
...,...,...,...,...
13442,4891,GrandTheftAuto(GTA),Irrelevant,⭐️ Toronto is the arts and culture capital of ...
13443,4359,CS-GO,Irrelevant,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
13444,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play...
13445,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.


In [ ]:
df_test.isnull().any()

,0
id,False
entity,False
sentiment,False
tweet,False


In [7]:
def prep_data(dataframe, trim, sentence_len=10):

  data = dataframe['tweet'].str.replace(r'http\S+|www\S+|\d+|[^A-Za-z\s#@]', '', regex=True)

  if trim == 1:
    data = data[data.str.split().str.len() >= sentence_len]
    label = dataframe['sentiment'][data.index].replace(['Positive','Neutral','Irrelevant', 'Negative'], [3,2,1,0])
    return data,label

  else:
    pass

In [8]:
data,label = prep_data(df,trim=True)
data_test, label_test = prep_data(df_test,trim=True)
data_join, label_join = prep_data(df_join,trim=True)

<ipython-input-7-0b81f3658fbc>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  label = dataframe['sentiment'][data.index].replace(['Positive','Neutral','Irrelevant', 'Negative'], [3,2,1,0])
<ipython-input-7-0b81f3658fbc>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  label = dataframe['sentiment'][data.index].replace(['Positive','Neutral','Irrelevant', 'Negative'], [3,2,1,0])
<ipython-input-7-0b81f3658fbc>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the ol

In [31]:
data_join

,tweet
0,im getting on borderlands and i will murder yo...
1,So I spent a few hours making something for fu...
2,RockHard La Varlope RARE POWERFUL HANDSOME JA...
3,that was the first borderlands session in a lo...
4,the biggest dissappoinment in my life came out...
...,...
13441,Good on Sony As much as I want to see the new ...
13442,Toronto is the arts and culture capital of Ca...
13443,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...
13444,Today sucked so its time to drink wine n play ...


In [32]:
label_join

,sentiment
0,3
1,3
2,2
3,3
4,0
...,...
13441,3
13442,1
13443,1
13444,3


### Project 1: simple sentiment analysis
predicting sentiment based on tweet

Data structure:
- tokenise tweets
- sentiment:
    - positive = 3
    - neutral = 2
    - irrelevant = 1
    - negative = 0

# SENTIMENT ANALYSIS WITH WHITESPACE TOKENIZED DATA & LOGISTIC REGRESSION (BASELINE)


In [10]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokens = data_join.apply(lambda x: tokenizer.tokenize(x, truncation=True, max_length=10))
input = np.stack(tokens.apply(lambda x: tokenizer.convert_tokens_to_ids(x)).to_numpy())


In [ ]:
input

array([[13280,  2033,  1113, ...,  1209,  3513,  1128],
       [ 1573,   146,  2097, ...,  1380,  1111,  4106],
       [ 2977,  3048,  2881, ...,  1162, 26547, 16941],
       ...,
       [  151, 18312,  1465, ...,  1660,  1146,  1157],
       [ 2421,  1185,  8468, ..., 23314, 17243,  1592],
       [ 2066,  2788,  1103, ...,  6603,  1110,  1176]])

In [11]:
y = label_join.to_numpy()[:6798]
y_test = label_join.to_numpy()[6799:]

x = input[:6798]
x_test = input[6799:]

In [50]:
input[:6798].shape

(6798, 10)

In [12]:
clf = LogisticRegression(random_state=0, max_iter=500)
clf.fit(x,y)

LogisticRegression(max_iter=500, random_state=0)

In [13]:
y_pred = clf.predict(x_test)

In [14]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [15]:
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.3717
Precision: 0.3182
Recall: 0.3717
F1 Score: 0.3354


# Accuracy ~ 37%

# SENTIMENT ANALYSIS WITH WORD EMBEDDING

In [23]:
vocab_size = tokenizer.vocab_size  # Get vocab size from tokenizer
embedding_dim = 50  # Choose embedding dimension
embeds = nn.Embedding(vocab_size, embedding_dim)

In [24]:
tensor_ids = torch.tensor(input, dtype=torch.long)
embedded = embeds(tensor_ids)
new = embedded.mean(dim=2).detach().numpy()

In [25]:
y = label_join.to_numpy()[:6798]
y_test = label_join.to_numpy()[6799:]

x = new[:6798]
x_test = new[6799:]

In [26]:
clf = LogisticRegression(max_iter=200)
clf.fit(x,y)

LogisticRegression(max_iter=200)

In [27]:
y_pred = clf.predict(x_test)

In [28]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [29]:
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.3651
Precision: 0.3224
Recall: 0.3651
F1 Score: 0.3339


# Accuracy ~ 37% same as with just tokenization

# scale features

In [30]:
from sklearn.preprocessing import StandardScaler

In [31]:
scaler = StandardScaler()

# Fit the scaler on the training data (x) and transform it
x_scaled = scaler.fit_transform(x)

# Transform the test data (x_test) based on the training data scaler
x_test_scaled = scaler.transform(x_test)

In [32]:
clf = LogisticRegression(max_iter=200)
clf.fit(x_scaled,y)

LogisticRegression(max_iter=200)

In [33]:
y_pred = clf.predict(x_test_scaled)

In [34]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [35]:
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.3651
Precision: 0.3313
Recall: 0.3651
F1 Score: 0.3372


# XGBOOST - performed best

In [36]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

In [37]:
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_clf.fit(x, y)
y_pred_xgb = xgb_clf.predict(x_test)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:59:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [38]:
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))

XGBoost Accuracy: 0.4975717439293598
              precision    recall  f1-score   support

           0       0.59      0.55      0.57       772
           1       0.40      0.44      0.42       312
           2       0.51      0.49      0.50       661
           3       0.43      0.47      0.45       520

    accuracy                           0.50      2265
   macro avg       0.48      0.49      0.48      2265
weighted avg       0.50      0.50      0.50      2265



In [39]:
xgb_clf = xgb.XGBClassifier(eval_metric='mlogloss')
xgb_clf.fit(x_scaled, y)
y_pred_xgb = xgb_clf.predict(x_test_scaled)

In [40]:
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))

XGBoost Accuracy: 0.4975717439293598
              precision    recall  f1-score   support

           0       0.59      0.55      0.57       772
           1       0.40      0.44      0.42       312
           2       0.51      0.49      0.50       661
           3       0.43      0.47      0.45       520

    accuracy                           0.50      2265
   macro avg       0.48      0.49      0.48      2265
weighted avg       0.50      0.50      0.50      2265



In [41]:
from sklearn.svm import SVC

In [45]:
svm_clf = SVC(kernel='rbf', C=1.0)
svm_clf.fit(x_scaled, y)
y_pred = svm_clf.predict(x_test_scaled)

In [46]:
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

SVM Accuracy: 0.40618101545253865
              precision    recall  f1-score   support

           0       0.55      0.52      0.53       772
           1       0.26      0.07      0.11       312
           2       0.38      0.41      0.39       661
           3       0.31      0.43      0.36       520

    accuracy                           0.41      2265
   macro avg       0.37      0.36      0.35      2265
weighted avg       0.40      0.41      0.39      2265

